In [ ]:
# 모두 실행하면 자동으로 크롤링

from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from Screenshot import Screenshot_Clipping
import random
import pyperclip

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox') #sandbox 사용 X
chrome_options.add_argument('--disable-dev-shm-usage')

chrome_path = 'C:/chromedriver/chromedriver.exe'
driver = webdriver.Chrome(chrome_path, chrome_options=chrome_options)
# driver1 = webdriver.Chrome(chrome_path, chrome_options=chrome_options)

In [ ]:
# 직접 입력 안될 시 copy 함수 사용

def copy_input(xpath, input):
    pyperclip.copy(input)
    driver.find_element_by_xpath(xpath).click()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)


email = '아이디'
pw = '비밀번호'

In [ ]:
def Crawling_Amazon(input_num, remain_num=22):
       
        url = f'https://worker.mturk.com/?sort=updated_desc&page_number={input_num}'

        driver.get(url)
        try :
        # 아이디 로그인
                driver.find_element_by_xpath('//*[@id="ap_email"]').send_keys('아이디')
                driver.find_element_by_xpath('//*[@id="ap_password"]').send_keys('비밀번호')
                time.sleep(random.randint(3,5))
                driver.find_element_by_xpath('//*[@id="signInSubmit"]').click()

                time.sleep(random.randint(3,5))

                url = f'https://worker.mturk.com/?sort=updated_desc&page_number={input_num}'
                driver.get(url)

                time.sleep(random.randint(3,5))
               
        except :
                url = f'https://worker.mturk.com/?sort=updated_desc&page_number={input_num}'
                driver.get(url)
                time.sleep(random.randint(3,5))


        for i in range(2, remain_num):
                # 생성 시간 (추후 index 값으로 설정) li 2 ~ 21
                created_time = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[1]/span[5]/span')
                time.sleep(random.randint(3,5))
                created_time.click()
                created_time = created_time.get_attribute('title')

                # 제목
                title = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[1]/span[2]').text

                # 신청인
                requester = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[1]/span[1]')
                requester = requester.get_attribute('title')

                # 리워드
                reward = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[1]/span[4]').text

                # HITs (목표 수량)
                hits = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[1]/span[3]').text

                try :
                        # 할당 시간 (Min 단위 숫자만 추출)
                        time_allotted = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[3]/div/div/div/div[2]/div/div[3]').text
                        time_allotted = time_allotted[time_allotted.index('Allotted')+9 : time_allotted.index('Min')-1]

                except :
                        # 할당 시간 (Min 단위 숫자만 추출)
                        time_allotted = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[3]/div/div/div/div[2]/div/div[3]').text
                        time_allotted = time_allotted[time_allotted.index('Allotted')+9 : time_allotted.index('Sec')-1]

                # 마감 날짜
                expires = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[3]/div/div/div/div[2]/div/div[4]/span')
                expires = expires.get_attribute('title')

                # 상세설명
                explain = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[3]/div/div/div/div[1]/p').text

                try :
                        # 조건 (변수명 : limit_숫자)

                        for j in range(1,10):
                                try:
                                        globals()[f'limit_{j}'] = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[3]/div/div[2]/div[2]/div[{j}]/div[1]/span').text
                                       
                                except:
                                        for k in range(1, j):
                                                # limit 1열
                                                limit_col1 = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[3]/div/div[2]/div[2]/div[{k}]/div[1]/span').text
                                                # limit 2열
                                                limit_col2 = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[3]/div/div[2]/div[2]/div[{k}]/div[2]').text
                                                # limit 3열
                                                limit_col3 = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[3]/div/div[2]/div[2]/div[{k}]/div[3]').text

                                                globals()[f'limit_{k}'] = [[limit_col1], [limit_col2], [limit_col3]]
                                                lim = j
                                        break
                       
                        lim_list = f'''{[globals()[f'limit_{s}'] for s in range(1, lim)]}'''
                       
                        Amazon_M_turk = {'created_time': created_time, 'title': title, 'requester': requester, 'hits':hits, 'reward':reward, 'time_allotted(Min)':time_allotted, 'expires':expires, 'explain':explain, 'limit': lim_list}
                                               
                        Amazon_M_turk_df = pd.DataFrame(Amazon_M_turk, index=[0])
                        Amazon_M_turk_df.set_index('created_time', inplace=True)
                        globals()[f'Amazon_M_turk_df_{i}'] = Amazon_M_turk_df

                except :        
                        Amazon_M_turk = {'created_time': created_time, 'title': title, 'requester': requester, 'hits':hits, 'reward':reward, 'time_allotted(Min)':time_allotted, 'expires':expires, 'explain':explain }
                        Amazon_M_turk_df = pd.DataFrame(Amazon_M_turk, index=[0])
                        Amazon_M_turk_df.set_index('created_time', inplace=True)
                        globals()[f'Amazon_M_turk_df_{i}'] = Amazon_M_turk_df

        df = pd.concat([globals()[f'Amazon_M_turk_df_{c}'] for c in range(2,remain_num)])


        return df

In [ ]:
def Crawling_Amazon_review(input_num, remain_num=23):
        i = 2
        url = f'https://worker.mturk.com/?sort=updated_desc&page_number={input_num}'

        driver.get(url)
        try :
        # 아이디 로그인
                copy_input('//*[@id="ap_email"]', email)
                time.sleep(random.randint(3,5))
                copy_input('//*[@id="ap_password"]', pw)
                time.sleep(random.randint(3,5))
                driver.find_element_by_xpath('//*[@id="signInSubmit"]').click()


                time.sleep(random.randint(5,10))

                url = f'https://worker.mturk.com/?sort=updated_desc&page_number={input_num}'
                driver.get(url)

                time.sleep(random.randint(10,15))
                
        except :
                url = f'https://worker.mturk.com/?sort=updated_desc&page_number={input_num}'
                driver.get(url)
                time.sleep(random.randint(3,5))
        

        while i < remain_num :
                try :
                        driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[4]/div/div/ol/li[{i}]/div[1]/span[6]/span/a').click()
                        time.sleep(random.randint(3,5))
                
                        # 생성 시간
                        created_time = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div/div[2]/div/div[4]/span[3]/span/span')
                        created_time = created_time.get_attribute('title')
                        created_time_name = created_time.replace('/', '.')
                        created_time_name = created_time_name.replace(':', '.')

                        # 타이틀
                        title = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div/div[1]/div/div/div/div[1]')
                        title = title.get_attribute('title')

                        iframe = driver.find_element_by_xpath(f'//*[@id="MainContent"]/div[5]/div/div/iframe')
                        url = iframe.get_attribute('src')
                        driver.get(url)
                        time.sleep(random.randint(3,5))

                        # review_text
                        review_text = driver.find_element_by_xpath('/html/body').text

                        ob = Screenshot_Clipping.Screenshot()
                        time.sleep(random.randint(3,5))
                        ob.full_Screenshot(driver, save_path=r'./Amazon_img/', image_name=f'Amazon_{created_time_name}_{title}.png')
                        
                        

                        Amazon_M_turk = {'created_time': created_time, 'title': title, 'review_text': review_text}
                        Amazon_M_turk_df = pd.DataFrame(Amazon_M_turk, index=[0])
                        Amazon_M_turk_df.set_index('created_time', inplace=True)
                        globals()[f'Amazon_M_turk_df_{i}'] = Amazon_M_turk_df
                        
                        url = f'https://worker.mturk.com/?sort=updated_desc&page_number={input_num}'
                        driver.get(url)
                        time.sleep(random.randint(3,5))
                        i += 1
                        
                except :
                        Amazon_M_turk = {'created_time': '11/2/2021 02:22pm', 'title': 'Classify Receipt'}
                        Amazon_M_turk_df = pd.DataFrame(Amazon_M_turk, index=[0])
                        Amazon_M_turk_df.set_index('created_time', inplace=True)
                        globals()[f'Amazon_M_turk_df_{i}'] = Amazon_M_turk_df
                        
                        url = f'https://worker.mturk.com/?sort=updated_desc&page_number={input_num}'
                        driver.get(url)
                        time.sleep(random.randint(3,5))
                        i += 1
        

        df = pd.concat([globals()[f'Amazon_M_turk_df_{c}'] for c in range(2, remain_num)])

        return df

In [ ]:
# df_list = pd.concat([Crawling_Amazon(x, 22) for x in range(30, 1, -1)])
# df_list = df_list.reset_index()
# df_list = df_list.drop_duplicates(['created_time', 'title'])
# df_list.set_index('created_time', inplace=True)

# df_list.to_csv('C:/Users/ooche/OneDrive - AROB/df_list.csv', encoding='utf-8-sig')

In [ ]:
# df_review = pd.concat([Crawling_Amazon_review(x, 23) for x in range(30, 1, -1)])
# df_review = df_review.reset_index()
# df_review = df_review.drop_duplicates(['created_time', 'title'])
# df_review.set_index('created_time', inplace=True)

# df_review.to_csv('C:/Users/ooche/OneDrive - AROB/df_review.csv', encoding='utf-8-sig')

In [ ]:
# df_merge = pd.merge(df_list, df_review, on = ['created_time', 'title'], how='outer')
# df_merge.set_index('created_time', inplace=True)
# df_merge.to_csv('C:/Users/ooche/OneDrive - AROB/df_merge.csv', encoding='utf-8-sig')

In [ ]:
# df5 = pd.read_csv('df5.csv')
# df5 = df5.reset_index()

while True:
    df5 = pd.read_csv('C:/Users/ooche/OneDrive - AROB/df_list.csv') # 기존 파일 불러오기
    df4 = Crawling_Amazon(1, 22) # 새로 크롤링한 파일
    df4 = df4.reset_index()
    df5 = pd.concat([df4, df5], ignore_index=False) # 새로 갱신된 파일과 merge
    df5 = df5.drop_duplicates(['created_time', 'title']) # 중복 제거
    df5 = df5.reset_index(drop=True)
    df5.set_index('created_time', inplace=True)

    df_review = pd.read_csv('C:/Users/ooche/OneDrive - AROB/df_review.csv') # 기존 파일 불러오기
    df10 = Crawling_Amazon_review(1, 23) # 새로 크롤링한 파일
    df10 = df10.reset_index()
    df10 = pd.concat([df10, df_review], ignore_index=False) # 새로 갱신된 파일과 merge
    df10 = df10.drop_duplicates(['created_time', 'title']) # 중복 제거
    df10 = df10.reset_index(drop=True)
    df10.set_index('created_time', inplace=True)

    df10.to_csv('C:/Users/ooche/OneDrive - AROB/df_review.csv', encoding='utf-8-sig') # 저장

    df5.to_csv('C:/Users/ooche/OneDrive - AROB/df_list.csv', encoding='utf-8-sig') # 저장

    df_list = pd.read_csv('C:/Users/ooche/OneDrive - AROB/df_list.csv') # 통합된 list csv 불러오기
    df_review = pd.read_csv('C:/Users/ooche/OneDrive - AROB/df_review.csv') # 통합된 review csv 불러오기
    df_review = df_review.drop('created_time', axis=1)
    df_merge = pd.merge(df_list, df_review, on = 'title', how='left') # list 기준 merge
    df_merge = df_merge.drop_duplicates(['created_time', 'title']) # 중복제거
    df_merge.set_index('created_time', inplace=True)
    df_merge.to_csv('C:/Users/ooche/OneDrive - AROB/df_merge_ing.csv', encoding='utf-8-sig') # 통합본 저장

    time.sleep(random.randint(3500,3700))